<a href="https://colab.research.google.com/github/terahidro2003/ID2223_finetuning/blob/main/ID2223_fine_tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
## Install Unsloth

In [2]:
%%capture

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

## Constants

In [3]:
import os 

if "google.colab" in str(get_ipython()):
  from google.colab import userdata
  TOKEN = userdata.get('HF_TOKEN')
else:
  !pip install python-dotenv
  from dotenv import load_dotenv
  load_dotenv()
  TOKEN = os.environ["HF_TOKEN"]

MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
hub_repo = "hellstone1918/Llama-3.2-3B-basic-lora-model"

Defaulting to user installation because normal site-packages is not writeable


## Specify Quantizied Model 

In [4]:
!pip install unsloth
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Defaulting to user installation because normal site-packages is not writeable
  Using cached unsloth-2025.11.4-py3-none-any.whl (358 kB)
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hellstone/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Adding LoRA Adapter

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # 0 is optimized by unsloth (fallback, if needed: 0.05)
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = True, # And LoftQ
)

/home/hellstone/.local/lib/python3.10/site-packages/peft/tuners/lora/config.py:730: UserWarning: `loftq_config` specified but will be ignored when `init_lora_weights` is not 'loftq'.
  warnings.warn("`loftq_config` specified but will be ignored when `init_lora_weights` is not 'loftq'.")
Unsloth 2025.11.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Data Preparation

## Download Dataset

In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

## Convert Dataset Format to HuggingFace's Generic Format

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True, )

## Comparison of Formats

In [8]:
dataset[5]["conversations"]

[{'content': 'How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?',
  'role': 'user'},
 {'content': 'Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.',
  'role': 'assistant'}]

In [9]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

# Training

In [10]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
import multiprocessing

sft_config = SFTConfig(
    output_dir       = "outputs",
    # batch size - number of training examples utilized in one iteration
    # epoch - one forward and backward pass of all training samples
    # epoch = iterations * batch_size
    # effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    per_device_train_batch_size = 2, # determines how many samples are loaded by the GPU at once (consumes more VRAM, but learning gets faster)
    gradient_accumulation_steps = 16,

    # warmup is needed at the beginning of the training as weights are unadapted to the new dataset. Therefore, we need to slowly increase the learning rate as we fine-tune the model.
    # - prevents large initial updates that could destabilize the model
    # - allows the model to explore the parameter space more effectively and transition smoothly from initial parameter space to regions of higher gradients
    # for large datasets or large amount of features, 5-10% of warmup steps is recommended. Since we use reletively small dataset (100k), it might be safe to go with 3-5% warmup steps (from overall steps)
    warmup_steps     = 150, # how many steps you use to slowly increase the learning rate from 0 up to the target learning rate

    # too many steps -> overfitting or model collapse, too few steps -> catastrophic forgetting
    max_steps        = 5000,

    learning_rate    = 2e-4,

    fp16             = not is_bfloat16_supported(),
    bf16             = is_bfloat16_supported(),
    logging_steps    = 1,
    optim            = "adamw_8bit",
    weight_decay     = 0.01, # adds a small penalty to large weights during training.
    lr_scheduler_type= "linear",
    seed             = 3407,

    # Checkpoint saving and pushing config
    push_to_hub = True,
    hub_model_id = hub_repo,
    hub_token=TOKEN,
    save_strategy="steps",
    save_steps       = 250,
    save_total_limit = 5,
    hub_strategy="all_checkpoints",

    report_to        = "none",

    dataset_num_proc = multiprocessing.cpu_count(),
    packing          = False,
)

trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = dataset,
    args               = sft_config,
    dataset_text_field = "text",
    max_seq_length     = max_seq_length,
    data_collator      = DataCollatorForSeq2Seq(tokenizer = tokenizer),
)

In [11]:
import os
import re
from huggingface_hub import HfApi, snapshot_download

def get_latest_hf_checkpoint(repo_id: str, cache_dir: str | None = None) -> str | None:
    """
    Returns local path to the latest checkpoint-* directory from a Hub repo,
    or None if the repo doesn't exist or has no checkpoints.
    """
    api = HfApi()
    try:
        api.repo_info(repo_id, repo_type="model")
    except:
        # Repo does not exist on the Hub -> first training ever
        return None

    # Repo exists – download files locally (or reuse cached snapshot)
    local_repo_path = snapshot_download(
        repo_id,
        repo_type="model",
        local_dir=cache_dir,                # e.g. "hf_repo_cache" or None
        local_dir_use_symlinks=False,
    )

    # Find checkpoint-* subdirectories
    ckpts: list[tuple[int, str]] = []
    for name in os.listdir(local_repo_path):
        full = os.path.join(local_repo_path, name)
        m = re.match(r"checkpoint-(\d+)", name)
        if m and os.path.isdir(full):
            step = int(m.group(1))
            ckpts.append((step, full))

    if not ckpts:
        # Repo exists but no checkpoints yet (maybe only final model)
        return None

    # Return path of checkpoint with highest step number
    _, latest_ckpt_path = max(ckpts, key=lambda t: t[0])
    return latest_ckpt_path


In [12]:
from unsloth.chat_templates import train_on_responses_only

#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)
latest_ckpt = get_latest_hf_checkpoint(hub_repo)
if latest_ckpt is None:
    print("No checkpoints on Hub – starting training from scratch.")
    trainer_stats = trainer.train()
else:
    print(f"Found checkpoint on Hub: {latest_ckpt}")
    trainer_stats = trainer.train(resume_from_checkpoint=latest_ckpt)

GPU = NVIDIA GeForce RTX 4070. Max memory = 11.994 GB.
3.07 GB of memory reserved.


The model is already on multiple devices. Skipping the move to device specified in `args`.


Found checkpoint on Hub: /home/hellstone/.cache/huggingface/hub/models--hellstone1918--Llama-3.2-3B-basic-lora-model/snapshots/83ccb953608d3c1dd39b21ed026367f4f14d8940/checkpoint-5000


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 2 | Total steps = 5,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5001,0.601800


## Show Stats

In [13]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

31.8118 seconds used for training.
0.53 minutes used for training.
Peak reserved memory = 4.863 GB.
Peak reserved memory for training = 1.793 GB.
Peak reserved memory % of max memory = 40.545 %.
Peak reserved memory for training % of max memory = 14.949 %.


In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "What is JVM?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

JVM (Java Virtual Machine) is a software layer that runs on top of the CPU's hardware. It is a component of Java development platform which is responsible to handle the execution of Java programs. It provides the following features: memory management, execution, runtime data access. JVM is designed to work with any environment in which Java programs can run, allowing them to be executed anywhere there is an JVM installed. It provides the runtime environment in which Java programs can run.<|eot_id|>


# Save the Model

In [16]:
!uv venv .venv
!source .venv/bin/activate --clear
!uv pip install "unsloth" "gguf" "protobuf" "sentencepiece" "mistral_common"

model.save_pretrained(MODEL_NAME + "_regular_lora")
tokenizer.save_pretrained(MODEL_NAME + "_regular_lora")

model.push_to_hub_gguf(hub_repo, tokenizer, quantization_method = "q4_k_m")

model.push_to_hub(hub_repo)
tokenizer.push_to_hub(hub_repo)

Using CPython 3.11.14
Creating virtual environment at: .venv
? A virtual environment already exists at `.venv`. Do you want to replace it? [y/n] › yes

hint: Use the `--clear` flag or set `UV_VENV_CLEAR=1` to skip this prompt
Audited 5 packages in 1.70s


Processing Files (1 / 1): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|  195MB /  195MB, 79.7MB/s  
New Data Upload: |                                                                                                                                                                  |  0.00B /  0.00B,  0.00B/s  


Saved model to https://huggingface.co/hellstone1918/Llama-3.2-3B-basic-lora-model


Processing Files (1 / 1): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17.2MB / 17.2MB,  0.00B/s  
New Data Upload: |                                                                                                                                                                  |  0.00B /  0.00B,  0.00B/s  
No files have been modified since last commit. Skipping to prevent empty commit.
[huggingface_hub.hf_api|WARNING]No files have been modified since last commit. Skipping to prevent empty commit.
